In [153]:
import tensorflow as tf
import numpy as np
import string

import os
from PIL import Image
#from cache import cache

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding, Dropout, LSTM, add
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import cv2

In [116]:
# Below is the path for the file "Flickr8k.token.txt" on your disk
filename = "captions.txt"
file = open(filename, 'r')
doc = file.read()

In [117]:
doc

'1000092795.jpg#0\tTwo young guys with shaggy hair look at their hands while hanging out in the yard .\n1000092795.jpg#1\tTwo young , White males are outside near many bushes .\n1000092795.jpg#2\tTwo men in green shirts are standing in a yard .\n1000092795.jpg#3\tA man in a blue shirt standing in a garden .\n1000092795.jpg#4\tTwo friends enjoy time spent together .\n10002456.jpg#0\tSeveral men in hard hats are operating a giant pulley system .\n10002456.jpg#1\tWorkers look down from up above on a piece of equipment .\n10002456.jpg#2\tTwo men working on a machine wearing hard hats .\n10002456.jpg#3\tFour men on top of a tall structure .\n10002456.jpg#4\tThree men on a large rig .\n1000268201.jpg#0\tA child in a pink dress is climbing up a set of stairs in an entry way .\n1000268201.jpg#1\tA little girl in a pink dress going into a wooden cabin .\n1000268201.jpg#2\tA little girl climbing the stairs to her playhouse .\n1000268201.jpg#3\tA little girl climbing into a wooden playhouse .\n10

In [118]:
descriptions = dict()
image_names=list()
for line in doc.split('\n'):
    # split line by white space
    tokens = line.split()
    image_id, image_desc = tokens[0], tokens[1:]
    image_id = image_id.split('.')[0]
    image_names.append(image_id+".jpg")
    
    image_desc = ' '.join(image_desc)
    if image_id not in descriptions:
        descriptions[image_id] = list()
    descriptions[image_id].append(image_desc)
print("completed")

completed


In [119]:
descriptions['1000092795']

['Two young guys with shaggy hair look at their hands while hanging out in the yard .',
 'Two young , White males are outside near many bushes .',
 'Two men in green shirts are standing in a yard .',
 'A man in a blue shirt standing in a garden .',
 'Two friends enjoy time spent together .']

In [120]:
train_images = image_names[0:25]
f = open("trainImages.txt","w")
f.write( str(train_images) )
f.close()

In [121]:
# prepare translation table for removing punctuation
translator = str.maketrans('', '', string.punctuation)
for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        # tokenize
        desc = desc.split()
        # convert to lower case
        desc = [word.lower() for word in desc]
        # remove punctuation from each token
        desc = [w.translate(translator) for w in desc]
        # remove hanging 's' and 'a'
        desc = [word for word in desc if len(word)>1]
        # remove tokens with numbers in them
        desc = [word for word in desc if word.isalpha()]
        # store as string
        desc_list[i] =  ' '.join(desc)

In [122]:
descriptions['1000092795']

['two young guys with shaggy hair look at their hands while hanging out in the yard',
 'two young white males are outside near many bushes',
 'two men in green shirts are standing in yard',
 'man in blue shirt standing in garden',
 'two friends enjoy time spent together']

In [123]:
vocabulary = set()
for key in descriptions.keys():
    [vocabulary.update(d.split()) for d in descriptions[key]]
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 107


In [124]:
train_descriptions = descriptions
f = open("descriptions.txt","w")
f.write( str(descriptions) )
f.close()

In [125]:
filename = 'trainImages.txt'
file = open(filename, 'r')
doc = file.read()
doc

"['1000092795.jpg', '1000092795.jpg', '1000092795.jpg', '1000092795.jpg', '1000092795.jpg', '10002456.jpg', '10002456.jpg', '10002456.jpg', '10002456.jpg', '10002456.jpg', '1000268201.jpg', '1000268201.jpg', '1000268201.jpg', '1000268201.jpg', '1000268201.jpg', '1000344755.jpg', '1000344755.jpg', '1000344755.jpg', '1000344755.jpg', '1000344755.jpg', '1000366164.jpg', '1000366164.jpg', '1000366164.jpg', '1000366164.jpg', '1000366164.jpg']"

In [126]:
train = list()
for line in doc.split(','):
    identifier = line.split('.')[0]
    train.append(identifier)
print('Dataset: %d' % len(train))

Dataset: 25


In [127]:
filename = 'descriptions.txt'
file = open(filename, 'r')
doc = file.read()
doc

"{'1000268201': ['child in pink dress is climbing up set of stairs in an entry way', 'little girl in pink dress going into wooden cabin', 'little girl climbing the stairs to her playhouse', 'little girl climbing into wooden playhouse', 'girl going into wooden building'], '10002456': ['several men in hard hats are operating giant pulley system', 'workers look down from up above on piece of equipment', 'two men working on machine wearing hard hats', 'four men on top of tall structure', 'three men on large rig'], '1000344755': ['someone in blue shirt and hat is standing on stair and leaning against window', 'man in blue shirt is standing on ladder cleaning window', 'man on ladder cleans the window of tall building', 'man in blue shirt and jeans on ladder cleaning windows', 'man on ladder cleans window'], '1000092795': ['two young guys with shaggy hair look at their hands while hanging out in the yard', 'two young white males are outside near many bushes', 'two men in green shirts are stan

In [132]:
# Below is the path for the file "Flickr8k.token.txt" on your disk
filename = "captions.txt"
file = open(filename, 'r')
doc = file.read()
descriptions = dict()

for line in doc.split('\n'):
    # split line by white space
    tokens = line.split()
    image_id, image_desc = tokens[0], tokens[1:]
    image_id = image_id.split('.')[0]
    image_names.append(image_id+".jpg")
    
    image_desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
    if image_id not in descriptions:
        descriptions[image_id] = list()
    descriptions[image_id].append(image_desc)
print("completed")

completed


In [134]:
train_descriptions = descriptions
train_descriptions

{'1000092795': ['startseq Two young guys with shaggy hair look at their hands while hanging out in the yard . endseq',
  'startseq Two young , White males are outside near many bushes . endseq',
  'startseq Two men in green shirts are standing in a yard . endseq',
  'startseq A man in a blue shirt standing in a garden . endseq',
  'startseq Two friends enjoy time spent together . endseq'],
 '10002456': ['startseq Several men in hard hats are operating a giant pulley system . endseq',
  'startseq Workers look down from up above on a piece of equipment . endseq',
  'startseq Two men working on a machine wearing hard hats . endseq',
  'startseq Four men on top of a tall structure . endseq',
  'startseq Three men on a large rig . endseq'],
 '1000268201': ['startseq A child in a pink dress is climbing up a set of stairs in an entry way . endseq',
  'startseq A little girl in a pink dress going into a wooden cabin . endseq',
  'startseq A little girl climbing the stairs to her playhouse . en

In [138]:
ixtoword = {}
wordtoix = {}
ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1
ix

43

In [165]:
model = VGG16(include_top=True, weights='imagenet')
transfer_layer = model.get_layer('fc2')
X = Dense(2048, activation = 'relu', name = 'fc0')(transfer_layer.output)
model_new = Model(inputs=model.input, outputs=X)

In [166]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
 all_desc = list()
 for key in descriptions.keys():
  [all_desc.append(d) for d in descriptions[key]]
 return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
 lines = to_lines(descriptions)
 return max(len(d.split()) for d in lines)
# determine the maximum sequence length

max_length = max_length(train_descriptions)
print('Max Description Length: %d' % max_length)

Max Description Length: 22


In [167]:

# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [168]:
# Load Glove vectors
glove_dir = 'glove'
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.50d.txt'), encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [169]:
vocabulary = set()
for key in descriptions.keys():
    [vocabulary.update(d.split()) for d in descriptions[key]]
print('Original Vocabulary Size: %d' % len(vocabulary))
vocab_size = len(vocabulary)

Original Vocabulary Size: 113


In [170]:
embedding_dim = 50
# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [171]:
# image feature extractor model
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# partial caption sequence model
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

# decoder (feed forward) model
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

# merge the two input models
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [172]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 22)           0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 22, 50)       5650        input_10[0][0]                   
__________________________________________________________________________________________________
dropout_7 (Dropout)             (None, 2048)         0           input_9[0][0]                    
__________________________________________________________________________________________________
dropout_8 

In [173]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [174]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [175]:
model.fit(x = [X1, X2], y = y)

NameError: name 'X1' is not defined

In [160]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [163]:
photo = 1000092795.jpg
greedySearch(photo)

SyntaxError: invalid syntax (<ipython-input-163-4b504342efbf>, line 1)

In [68]:
# Create a list of all the training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)

# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 2
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print('preprocessed words %d ' % len(vocab))

preprocessed words 42 
